In [1]:
import pandas as pd

In [ ]:
df_meta = pd.read_csv("dash_dataset.csv")[["meta_id", "sex", "age", "BMI", "study", "length_of_fasting", "length_of_fasting_cat", "age_cat", "BMI_cat"]].drop_duplicates()
df_long = (
    pd.read_csv("dash_dataset.csv")
    .drop(columns=["length_of_fasting_cat", "sex", "age_cat", 
                   "BMI_cat", "age", "length_of_fasting", "BMI", "study"])
    .melt(
        id_vars = ["meta_id"]
    )
    .assign(    
            # Détecter "Pre", "Pobst" ou le signe delta (Δ ou 'Δ') même sans unités dans le nom de variable
            timepoint = lambda x: (
                x.variable.str.extract(r"(Pre|Post|Δ|\u0394)", expand=False)
            ),
            variable = lambda x: (
                x.variable
                 .str.replace(r" Pre$| Post$| Δ$| Δ$", "", regex=True)
            ),
    )
    .merge(df_meta, on="meta_id", how="left")
)

In [31]:
import sys
import numpy as np

pivot_df = (
    df_long
     .pivot_table(
        index = ["meta_id", "sex", "age_cat", "BMI_cat", "length_of_fasting_cat"], 
        columns = ["timepoint", "variable"], 
        values = "value"
    )
     .reset_index()
)#.to_csv("dash_dataset_wide.csv")

memory_usage_bytes = pivot_df.memory_usage(deep=True).sum()
memory_usage_mb = memory_usage_bytes / (1024 ** 2)

print(f"Taille du DataFrame pivoté : {memory_usage_mb:.2f} Mo")

Taille du DataFrame pivoté : 3.79 Mo


In [32]:
pivot_df

timepoint         meta_id sex                     age_cat  \
variable                                                    
0              1-marbella   F    Middle age (35-64 years)   
1            1-uberlingen   M    Middle age (35-64 years)   
2             10-marbella   F    Older adults (≥65 years)   
3           10-uberlingen   F    Middle age (35-64 years)   
4          100-uberlingen   F  Young adults (18-34 years)   
...                   ...  ..                         ...   
2445       995-uberlingen   F    Middle age (35-64 years)   
2446       996-uberlingen   M    Older adults (≥65 years)   
2447       997-uberlingen   F    Middle age (35-64 years)   
2448       998-uberlingen   M    Middle age (35-64 years)   
2449       999-uberlingen   F    Middle age (35-64 years)   

timepoint                       BMI_cat length_of_fasting_cat      Post  \
variable                                                      ALP [U/L]   
0          Overweight (25.0–29.9 kg/m²)              3-7 days       NaN   
1                   Obesity (≥30 kg/m²)              3-7 days      49.0   
2                   Obesity (≥30 kg/m²)            13-17 days       NaN   
3                   Obesity (≥30 kg/m²)              3-7 days      54.0   
4                Normal (18–24.9 kg/m²)             8-12 days      46.0   
...                                 ...                   ...       ...   
2445       Overweight (25.0–29.9 kg/m²)            13-17 days      59.0   
2446                Obesity (≥30 kg/m²)            13-17 days      31.0   
2447                Obesity (≥30 kg/m²)             8-12 days      53.0   
2448                Obesity (≥30 kg/m²)            13-17 days      56.0   
2449                Obesity (≥30 kg/m²)            13-17 days      59.0   

timepoint                                                  ...  \
variable  BMI [kg/m²] CRP hs [mg/L] Ca [mg/dL] DBP [mmHg]  ...   
0           27.254475           NaN    8.50000       63.0  ...   
1           30.573510          1.70    9.45888        NaN  ...   
2           33.242932           NaN   10.00000       70.0  ...   
3                 NaN         13.80    9.21840        NaN  ...   
4           18.585859          0.62    9.73944       77.0  ...   
...               ...           ...        ...        ...  ...   
2445        23.380999          3.40    9.85968       74.0  ...   
2446        29.022469          4.30    9.85968       80.0  ...   
2447              NaN          1.10   10.26048        NaN  ...   
2448        28.081633          3.20    9.97992       81.0  ...   
2449              NaN          1.50    8.97792       94.0  ...   

timepoint                Δ                                        \
variable  leukocytes [G/L] physical activity [min/day] quick [%]   
0                      NaN                         0.0       NaN   
1                      NaN                         NaN      -5.0   
2                      NaN                        60.0       NaN   
3                      1.3                         NaN      -5.0   
4                     -4.5                         NaN     -21.0   
...                    ...                         ...       ...   
2445                  -2.2                         NaN      -9.0   
2446                  -2.0                         NaN     -11.0   
2447                  -1.2                         NaN     -16.0   
2448                  -2.8                         NaN     -13.0   
2449                  -0.7                         NaN     -32.0   

timepoint                                                       \
variable  sleep quality [0-10] thrombocytes [G/L] urea [mg/dL]   
0                          0.0                NaN         -6.0   
1                          NaN                NaN          5.3   
2                          3.0                NaN        -14.0   
3                          NaN              -28.0         -0.2   
4                          NaN               84.0         -9.7   
...                     

In [16]:
vars_to_drop = ["ketonuria [mg/dL]", "physical activity [min/day]", "sleep quality [0-10]", "PWB [0-10]", "PSA total [ng/mL]", "INR [-].query"]
(pd.read_csv("dash_dataset.csv")
    .drop(columns=["length_of_fasting_cat", "sex", "age_cat", 
                   "BMI_cat", "age", "length_of_fasting", "BMI", "study"])
    .melt(
        id_vars = ["meta_id"]
    )
    .assign(    
            # Détecter "Pre", "Pobst" ou le signe delta (Δ ou 'Δ') même sans unités dans le nom de variable
            timepoint = lambda x: (
                x.variable.str.extract(r"(Pre|Post|Δ|\u0394)", expand=False)
            ),
            variable = lambda x: (
                x.variable
                 .str.replace(r" Pre$| Post$| Δ$| Δ$", "", regex=True)
            ),
    )
    .query("variable not in @vars_to_drop")
    
    .merge(df_meta, on="meta_id", how="left")
).to_csv("df_long.csv")

In [14]:
for col in pd.read_csv("dash_dataset.csv").columns:
    print(col   )

meta_id
length_of_fasting_cat
sex
age_cat
BMI_cat
BMI
study
age
length_of_fasting
ALP [U/L] Pre
ALP [U/L] Post
ALP [U/L] Δ
BMI [kg/m²] Pre
BMI [kg/m²] Post
BMI [kg/m²] Δ
CRP hs [mg/L] Pre
CRP hs [mg/L] Post
CRP hs [mg/L] Δ
Ca [mg/dL] Pre
Ca [mg/dL] Post
Ca [mg/dL] Δ
DBP [mmHg] Pre
DBP [mmHg] Post
DBP [mmHg] Δ
ESR 1H [mm/h] Pre
ESR 1H [mm/h] Post
ESR 1H [mm/h] Δ
ESR 2H [mm/h] Pre
ESR 2H [mm/h] Post
ESR 2H [mm/h] Δ
EWB [0-10] Pre
EWB [0-10] Post
EWB [0-10] Δ
FLI Pre
FLI Post
FLI Δ
GFR [mL/min/1.73m²] Pre
GFR [mL/min/1.73m²] Post
GFR [mL/min/1.73m²] Δ
GGT [U/L] Pre
GGT [U/L] Post
GGT [U/L] Δ
GOT AST [U/L] Pre
GOT AST [U/L] Post
GOT AST [U/L] Δ
GPT ALT [U/L] Pre
GPT ALT [U/L] Post
GPT ALT [U/L] Δ
HBA1C [%] Pre
HBA1C [%] Post
HBA1C [%] Δ
HBA1C [mmol/mol] Pre
HBA1C [mmol/mol] Post
HBA1C [mmol/mol] Δ
HDL [mg/dL] Pre
HDL [mg/dL] Post
HDL [mg/dL] Δ
HR [/min] Pre
HR [/min] Post
HR [/min] Δ
INR [-] Pre
INR [-] Post
INR [-] Δ
K [mmol/L] Pre
K [mmol/L] Post
K [mmol/L] Δ
LDL HDL ratio [-] Pre
LDL HD

In [ ]:
import pandas as pd
df_long = pd.read_csv("df_long.csv", index_col=0, parse_dates=False)

In [4]:
df_long

,meta_id,variable,value,timepoint,sex,age,BMI,study,length_of_fasting,length_of_fasting_cat,age_cat,BMI_cat
0,1-marbella,ALP [U/L],NaN,Pre,F,53.67,28.279721,marbella,7.0,3-7 days,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²)
1,1-uberlingen,ALP [U/L],53.0,Pre,M,61.48,31.770459,uberlingen,5.0,3-7 days,Middle age (35-64 years),Obesity (≥30 kg/m²)
2,10-marbella,ALP [U/L],NaN,Pre,F,66.71,35.626233,marbella,17.0,13-17 days,Older adults (≥65 years),Obesity (≥30 kg/m²)
3,10-uberlingen,ALP [U/L],55.0,Pre,F,45.23,NaN,uberlingen,8.0,3-7 days,Middle age (35-64 years),Obesity (≥30 kg/m²)
4,100-uberlingen,ALP [U/L],50.0,Pre,F,26.35,19.834711,uberlingen,9.0,8-12 days,Young adults (18-34 years),Normal (18–24.9 kg/m²)
...,...,...,...,...,...,...,...,...,...,...,...,...
374845,995-uberlingen,weight [kg],-4.6,Δ,F,58.68,25.271203,uberlingen,16.0,13-17 days,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²)
374846,996-uberlingen,weight [kg],-7.2,Δ,M,76.11,31.346849,uberlingen,16.0,13-17 days,Older adults (≥65 years),Obesity (≥30 kg/m²)
374847,997-uberlingen,weight [kg],NaN,Δ,F,53.82,NaN,uberlingen,10.0,8-12 days,Middle age (35-64 years),Obesity (≥30 kg/m²)
374848,998-uberlingen,weight [kg],-7.0,Δ,M,59.84,30.367347,uberlingen,16.0,13-17 days,Middle age (35-64 years),Obesity (≥30 kg/m²)


In [2]:
df_long#.to_csv("df_long.csv")

NameError: name 'df_long' is not defined

In [ ]:
px.scatter(df_long, x="length_of", y="BMI", color="sex")

array(['ALP [U/L]', 'BMI [kg/m²]', 'CRP hs [mg/L]', 'Ca [mg/dL]',
       'DBP [mmHg]', 'ESR 1H [mm/h]', 'ESR 2H [mm/h]', 'EWB [0-10]',
       'FLI', 'GFR [mL/min/1.73m²]', 'GGT [U/L]', 'GOT AST [U/L]',
       'GPT ALT [U/L]', 'HBA1C [%]', 'HBA1C [mmol/mol]', 'HDL [mg/dL]',
       'HR [/min]', 'INR [-]', 'K [mmol/L]', 'LDL HDL ratio [-]',
       'LDL [mg/dL]', 'MCH [pg]', 'MCHC [g/dL]', 'MCV [fL]', 'Mg [mg/dL]',
       'Na [mmol/L]', 'PSA total [ng/mL]', 'PTT [s]', 'PWB [0-10]',
       'SBP [mmHg]', 'TC [mg/dL]', 'TG [mg/dL]', 'TSH [µU/mL]', 'WC [cm]',
       'creatinine [mg/dL]', 'erythrocytes [T/L]', 'glucose [mg/dL]',
       'hematocrit [%]', 'hemoglobin [g/dL]', 'hunger [0-10]',
       'ketonuria [mg/dL]', 'leukocytes [G/L]',
       'physical activity [min/day]', 'quick [%]', 'sleep quality [0-10]',
       'thrombocytes [G/L]', 'urea [mg/dL]', 'uric acid [mg/dL]',
       'vitamin B12 [pg/mL]', 'vitamin D [ng/mL]', 'weight [kg]'],
      dtype=object)

In [35]:
BMI = 27
age =55
sex = "M"
parameter = 'ALP [U/L]'
Pre_value = 70
df_pat =(
    df_long
    .query("BMI.between(@BMI - 2, @BMI + 2) and age.between(@age - 7, @age + 7) and sex == @sex")

    
)
df_pat_param = df_pat.query("variable == @parameter")
df_pat

,meta_id,length_of_fasting_cat,sex,age_cat,BMI_cat,age,length_of_fasting,BMI,study,variable,value,timepoint
13,101-marbella,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),53.28,7.0,25.465372,marbella,ALP [U/L],NaN,Pre
52,1042-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),52.69,13.0,26.612713,uberlingen,ALP [U/L],72.0,Pre
228,1203-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),54.20,9.0,28.468628,uberlingen,ALP [U/L],51.0,Pre
247,1222-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),55.65,11.0,28.395062,uberlingen,ALP [U/L],74.0,Pre
253,1228-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),59.17,11.0,26.948463,uberlingen,ALP [U/L],54.0,Pre
...,...,...,...,...,...,...,...,...,...,...,...,...
374704,861-uberlingen,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),53.76,6.0,26.153990,uberlingen,weight [kg],-3.1,Δ
374711,868-uberlingen,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),49.08,8.0,27.598650,uberlingen,weight [kg],-4.8,Δ
374808,96-marbella,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),57.91,10.0,27.382125,marbella,weight [kg],-6.0,Δ
374836,987-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),56.64,11.0,26.258270,uberlingen,weight [kg],-4.7,Δ


In [45]:
df_pat.variable.unique()

array(['ALP [U/L]', 'BMI [kg/m²]', 'CRP hs [mg/L]', 'Ca [mg/dL]',
       'DBP [mmHg]', 'ESR 1H [mm/h]', 'ESR 2H [mm/h]', 'EWB [0-10]',
       'FLI', 'GFR [mL/min/1.73m²]', 'GGT [U/L]', 'GOT AST [U/L]',
       'GPT ALT [U/L]', 'HBA1C [%]', 'HBA1C [mmol/mol]', 'HDL [mg/dL]',
       'HR [/min]', 'INR [-]', 'K [mmol/L]', 'LDL HDL ratio [-]',
       'LDL [mg/dL]', 'MCH [pg]', 'MCHC [g/dL]', 'MCV [fL]', 'Mg [mg/dL]',
       'Na [mmol/L]', 'PSA total [ng/mL]', 'PTT [s]', 'PWB [0-10]',
       'SBP [mmHg]', 'TC [mg/dL]', 'TG [mg/dL]', 'TSH [µU/mL]', 'WC [cm]',
       'creatinine [mg/dL]', 'erythrocytes [T/L]', 'glucose [mg/dL]',
       'hematocrit [%]', 'hemoglobin [g/dL]', 'hunger [0-10]',
       'ketonuria [mg/dL]', 'leukocytes [G/L]',
       'physical activity [min/day]', 'quick [%]', 'sleep quality [0-10]',
       'thrombocytes [G/L]', 'urea [mg/dL]', 'uric acid [mg/dL]',
       'vitamin B12 [pg/mL]', 'vitamin D [ng/mL]', 'weight [kg]'],
      dtype=object)

In [78]:
df_pat.query("variable == 'BMI [kg/m²]'")

,meta_id,length_of_fasting_cat,sex,age_cat,BMI_cat,age,length_of_fasting,BMI,study,variable,value,timepoint
7363,101-marbella,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),53.28,7.0,25.465372,marbella,BMI [kg/m²],25.465372,Pre
7402,1042-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),52.69,13.0,26.612713,uberlingen,BMI [kg/m²],26.612713,Pre
7578,1203-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),54.20,9.0,28.468628,uberlingen,BMI [kg/m²],28.468628,Pre
7597,1222-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),55.65,11.0,28.395062,uberlingen,BMI [kg/m²],28.395062,Pre
7603,1228-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),59.17,11.0,26.948463,uberlingen,BMI [kg/m²],26.948463,Pre
...,...,...,...,...,...,...,...,...,...,...,...,...
14554,861-uberlingen,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),53.76,6.0,26.153990,uberlingen,BMI [kg/m²],-0.967510,Δ
14561,868-uberlingen,3-7 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),49.08,8.0,27.598650,uberlingen,BMI [kg/m²],-1.603796,Δ
14658,96-marbella,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),57.91,10.0,27.382125,marbella,BMI [kg/m²],-1.791633,Δ
14686,987-uberlingen,8-12 days,M,Middle age (35-64 years),Overweight (25.0–29.9 kg/m²),56.64,11.0,26.258270,uberlingen,BMI [kg/m²],-1.388233,Δ


In [72]:
import statsmodels.formula.api as smf   

variables = ["ALP [U/L]", "LDL [mg/dL]",
              "TG [mg/dL]", "CRP hs [mg/L]",
             "TC [mg/dL]", "SBP [mmHg]",
             "DBP [mmHg]", "glucose [mg/dL]", "HDL [mg/dL]", 
             "GOT AST [U/L]", "GPT ALT [U/L]", "LDL [mg/dL]", "weight [kg]"]
tables = []
for var in variables:
    df_reg = (df_long
            .query("variable == @var").dropna()
            .pivot_table(
                index=["meta_id", "sex", 
                        "BMI", "age"], 
                columns="timepoint", values="value")
            
            ).reset_index()
    formula = f"Post ~ Pre + age + BMI + sex " if var != "weight [kg]" else "Post ~ Pre + age + sex "
    model = smf.ols('Post ~ Pre + age + BMI + sex ', data=df_reg).fit()
    table_res = model.summary2().tables[1].assign(variable=var).reset_index().set_index("variable").reset_index()
    tables.append(table_res)

In [73]:
df_stats =pd.concat(tables)

In [75]:
df_stats.to_csv("df_stats.csv")

In [59]:
table_res#.loc["P>|t|"]

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Intercept,4.216189,1.574446,2.677887,0.007517,1.127008,7.305369
sex[T.M],-0.161901,0.508362,-0.318477,0.750182,-1.159345,0.835542
Pre,0.765966,0.013789,55.547345,0.000000,0.738910,0.793022
age,0.065570,0.018570,3.530884,0.000431,0.029133,0.102006
BMI,0.170675,0.043740,3.902046,0.000101,0.084854,0.256495
